<a href="https://colab.research.google.com/github/jchen6727/tutorials/blob/master/Iab2/Iab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install neuron

In [ ]:
from neuron import h, gui
import matplotlib
%matplotlib inline
# see what is in h
dir(h)

In [ ]:
# create a simple model cell as a NEURON Section
soma = h.Section(name='soma')
# call psection() to see details of the Section
soma.psection()
# probe objects with the dir() function
dir(soma)

In [ ]:
# access sections by their name and segments
print("type(soma) = {}".format(type(soma)))
print("type(soma(0.5)) = {}".format(type(soma(0.5))))
# set the number of segments in soma
soma.nseg = 3
soma(0.1)
soma(0.5)
soma(0.9)

In [ ]:
# create a dendritic section 'dend' and connect it to the '1' end of the soma
dend = h.Section(name='dend')
dend.connect(soma(1))
# check the connection by using h.psections()
h.psection(sec=dend)
# confirm with NEURON's topology()
h.topology()
# dend[0] connected to soma[1]

In [ ]:
soma.L = soma.diam = 12.6157
dend.L = 180
dend.diam = 1
dend.nseg = 11

print("Surface area of soma = {}".format(h.area(0.5, sec=soma)))

In [ ]:
# use h.allsec() to iterate over all sections
for sec in h.allsec():
    sec.Ra = 100 # Axial resistance in Ohm * cm
    sec.cm = 1 # Membrane capacitance in uFarads / cm^2

In [ ]:
# insert active Hodgkin-Huxley current in the soma
soma.insert('hh')

# insert passive leak current in dendrite
dend.insert('pas')

In [ ]:
# accessing segment variables
print(dend(0.5).pas.g)
print(dend(0.5).g_pas)
mech = dend(0.5).pas
print(dir(mech))
print(mech.g)

In [ ]:
# set variable g of ALL segments in the section using section.var_mech
dend.g_pas = 0.001
print(dend(0.1).pas.g)
print(dend(0.9).pas.g)


In [ ]:
shape_window = h.PlotShape()
shape_window.exec_menu('Show Diam')

In [ ]:
soma.gnabar_hh = 0.12 # Sodium conductance in S/cm^2
soma.gkbar_hh = 0.036 #
soma.gl_hh = 0.0003
soma.el_hh = -54.3

dend.g_pas = 0.001
dend.e_pas = -65

In [ ]:
# insert a current clam (IClamp) to induce some membrane dynamics.
stim = h.IClamp(dend(1.0))


In [ ]:
dir(stim)

stim.amp = 0.1
stim.delay = 20
stim.dur = 3

In [ ]:
v_vec_soma = h.Vector()
v_vec_dend = h.Vector()
t_vec = h.Vector()
v_vec_soma.record(soma(0.5)._ref_v)
v_vec_dend.record(dend(1.0)._ref_v)
t_vec.record(h._ref_t)

In [ ]:
h.tstop = 40
h.load_file('stdrun.hoc')
h.run()

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(8,4))
plt.plot(t_vec, v_vec_soma, 'b', label='soma')
plt.plot(t_vec, v_vec_dend, 'r', label='dend')
plt.xlabel('time (ms)')
plt.ylabel('mV')
plt.show()